# Tutorial with examples

In [1]:
from cmfa.fluxomics_data.reaction import Reaction
from cmfa.fluxomics_data.reaction_network import ReactionNetwork
from cmfa.data_preparation import import_fluxomics_dataset_from_json


In [2]:
import os

CUR_DIR = os.getcwd()
TEST_DIR = f'{CUR_DIR}/../data/test_data'

TEST_DIR

'/Users/te/Library/CloudStorage/OneDrive-Personal/Biosustain/cmfa/cmfa/../data/test_data'

In [3]:
model = import_fluxomics_dataset_from_json(f'{TEST_DIR}/model.json')

In [4]:
model.reaction_network.reactions

{Reaction id=R1, name=R1,stoichiometry={'A': {abc: -1.0}, 'B': {abc: 1.0}},direction=False, ,
 Reaction id=R2, name=R2,stoichiometry={'B': {abc: -1.0}, 'D': {abc: 1.0}},direction=True, ,
 Reaction id=R3, name=R3,stoichiometry={'B': {abc: -1.0}, 'C': {bc: 1.0}, 'E': {a: 1.0}},direction=False, ,
 Reaction id=R4, name=R4,stoichiometry={'B': {abc: -1.0}, 'C': {de: -1.0}, 'D': {bcd: 1.0}, 'E': {a: 1.0, e: 1.0}},direction=False, ,
 Reaction id=R5, name=R5,stoichiometry={'D': {abc: -1.0}, 'F': {abc: 1.0}},direction=False, }

In [5]:
print(model.reaction_network.reaction_adjacency_matrix())

compound           A       B     C           D               E           F
pattern          abc     abc    bc  de     abc   bcd         a     e   abc
compound pattern                                                          
A        abc      []    [R1]    []  []      []    []        []    []    []
B        abc      []      []  [R3]  []  [R2_f]  [R4]  [R3, R4]    []    []
C        bc       []      []    []  []      []    []        []    []    []
         de       []      []    []  []      []  [R4]        []  [R4]    []
D        abc      []  [R2_r]    []  []      []    []        []    []  [R5]
         bcd      []      []    []  []      []    []        []    []    []
E        a        []      []    []  []      []    []        []    []    []
         e        []      []    []  []      []    []        []    []    []
F        abc      []      []    []  []      []    []        []    []    []


In [6]:
df = model.reaction_network.reaction_adjacency_matrix()
column_data = df[('F', 'abc')]

# Step 2: Filter for non-empty cells
# Assuming non-empty cells contain lists with at least one element, and empty cells contain empty lists.
non_empty_cells = column_data[column_data.apply(lambda x: len(x)>0)]

# Step 3: Retrieve the non-empty values and their corresponding rows
for index, value in non_empty_cells.items():
    print(f"Row: {index}, Value: {value}")

Row: ('D', 'abc'), Value: ['R5']


In [43]:
import pandas as pd
import re
from collections import deque

def _extract_atom_pattern(compound_pattern: str):
    """Extract the atom pattern from a compound pattern string.

    Parameters
    ----------
    compound_pattern : str
        The compound pattern string, e.g., "Glu(abcde)".

    Returns
    -------
    str
        The extracted atom pattern, e.g., "abcde".
    """
    match = re.search(r'(.+)\((.+)\)', compound_pattern)
    if match:
        compound, pattern = match.groups()
        return compound, pattern 
    else:
        raise ValueError(f"No atom pattern found in the input: {compound}")

def decompose_network(
    target_compound: str, reaction_network: ReactionNetwork
) -> pd.DataFrame:
    

    MAM = reaction_network.reaction_adjacency_matrix()
    queue = deque([target_compound])
    visited = set()
    emu_maps = {}

    while queue:
        cur_emu = queue.popleft()
        print(cur_emu)
        if cur_emu in visited:
            continue
        
        visited.add(cur_emu)

        # Pattern matching
        cpd, pat = cur_emu[0], cur_emu[1]
        emu_size = len(pat)
        if emu_size not in emu_maps:
            emu_maps[emu_size] = {}
        
        # Find the reaction that is participated in the current emu
        col = MAM[cur_emu]
        reactants = col[col.apply(lambda x: len(x)>0)]
        queue.append(tuple(reactants.index))
        print(queue)
        for next_emu in reactants:
            emu_maps[emu_size].setdefault(emu_size, []).append(next_emu)

            
    return emu_maps

In [44]:
decompose_network(('F','abc'), model.reaction_network)

deque([(('D', 'abc'),)])
['R5']


IndexError: tuple index out of range

In [28]:
target_compound = pd.MultiIndex.from_tuples([('ac','abc'), ('ac','ad')], name=('a', 'b'))
target_compound[1]['a']

TypeError: tuple indices must be integers or slices, not str